In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statistics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from  sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from statistics import mode 
from scipy import stats
data = pd.read_pickle('C:/Users/Pranita R. Shinde/Desktop/Miss Pranita/Internships/ANZ_July2021/ANZ_2.pkl')
data[0:5]


,status,card_present_flag,account,long_lat,txn_description,merchant_id,first_name,balance,date,gender,...,extraction,amount,transaction_id,customer_id,merchant_long_lat,movement,month,dayofweek,hour,category
0,authorized,1.0,ACC-1598451071,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,Diana,35.39,2018-08-01,F,...,01:01:15,16.25,a623070bfead4541a6b0fff8a09e706c,CUS-2487424745,153.38 -27.99,debit,8,2,1,Purchase
1,authorized,0.0,ACC-1598451071,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,Diana,21.20,2018-08-01,F,...,01:13:45,14.19,13270a2a902145da9db4c951e04b51b9,CUS-2487424745,151.21 -33.87,debit,8,2,1,Purchase
2,authorized,1.0,ACC-1222300524,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,Michael,5.71,2018-08-01,M,...,01:26:15,6.42,feb79e7ecd7048a5a36ec889d1a94270,CUS-2142601169,151.21 -33.87,debit,8,2,1,Purchase
3,authorized,1.0,ACC-1037050564,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,Rhonda,2117.22,2018-08-01,F,...,01:38:45,40.90,2698170da3704fd981b15e64a006079e,CUS-1614226872,153.05 -26.68,debit,8,2,1,Purchase
4,authorized,1.0,ACC-1598451071,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,Diana,17.95,2018-08-01,F,...,01:51:15,3.25,329adf79878c4cf0aeb4188b4691c266,CUS-2487424745,153.44 -28.06,debit,8,2,1,Purchase


In [38]:
pd.DataFrame({"Columns":data.columns})

,Columns
0,status
1,card_present_flag
2,account
3,long_lat
4,txn_description
5,merchant_id
6,first_name
7,balance
8,date
9,gender


In [39]:
## Feature


In [40]:
#Target variable: Annual salary

In [41]:
#Find unique customer id and make dataframe out of it.

In [42]:
salary_df = pd.DataFrame({'cust_id':data.customer_id.unique()})
salary_df.head()

,cust_id
0,CUS-2487424745
1,CUS-2142601169
2,CUS-1614226872
3,CUS-2688605418
4,CUS-4123612273


In [43]:
# To find the annual salary, w e will look at how frequently the salaries are paid and hence the salary amount each time.

In [44]:
df_freq = []
df_amount= [] 
for cust in range(len(salary_df)):
    salary = data.loc[(data.customer_id==salary_df.cust_id[cust]) & (data.txn_description=="PAY/SALARY"),['date','amount']].groupby('date',as_index = False).sum() 
    count = len(salary)
    if(count ==0):
        df_amount.append(np.nan)
        df_freq.append(np.nan)
    else:
        days_betn_payment = []
        for date in range(len(salary)-1):
            days_betn_payment.append((salary.date[date+1]-salary.date[date]).days)
        df_freq.append(max(days_betn_payment))
        df_amount.append(salary.amount.mode())
salary_df['salary_freq'] = df_freq
salary_df['salary_amount'] = df_amount
salary_df['annual_salart'] = salary_df['salary_amount']/salary_df['salary_freq']*365.25
salary_df.head()

,cust_id,salary_freq,salary_amount,annual_salart
0,CUS-2487424745,7,0 1013.67 dtype: float64,0 52891.8525 dtype: float64
1,CUS-2142601169,7,0 1002.13 dtype: float64,0 52289.711786 dtype: float64
2,CUS-1614226872,7,0 892.09 dtype: float64,0 46547.981786 dtype: float64
3,CUS-2688605418,14,0 2320.3 dtype: float64,0 60534.969643 dtype: float64
4,CUS-4123612273,7,0 1068.04 dtype: float64,0 55728.801429 dtype: float64


In [45]:
#Prdictive analysis

In [46]:
# unique cust_id
unique_id = data.customer_id.unique()
len(unique_id)

100

In [47]:
# first variable effecting the annual salarAverage number f weekly transactionsy: 

In [48]:
avg_of_weekly_trans = []
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'date']
    avg_of_weekly_trans.append(round(len(avg_of_weekly_trans)/array.nunique()*7))
    avg_of_weekly_trans[0:10]

In [49]:
# second variable effecting the annual salary maximum

In [50]:
max_amount = []
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'amount']
    max_amount.append(max(array))
max_amount[0:4]

[1452.21, 2349.55, 892.09, 2320.3]

In [51]:
#thiednumber of transactions above $100

In [52]:
 no_large_trans =[]
for amt in unique_id:
    array = data.loc[data.customer_id ==id, 'amount']
    count = 0
    for amt in array:
        if(amt>100):
            count+=1
    no_large_trans.append(count)
no_large_trans[0:5]

[14, 14, 14, 14, 14]

In [53]:
#aversge trns amont
avg_trn_amount = []
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'amount']
    avg_trn_amount.append(np.mean(array))
avg_trn_amount[0:4]

[45.34877162629756, 78.20610561056101, 74.46501930501928, 159.3041860465116]

In [54]:
#Median balance

In [55]:
median_balance = []
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'balance']
    median_balance.append(array.median())
median_balance[0:4]

[1580.4, 1132.66, 3618.5, 5616.63]

In [56]:
#state of resedence

In [57]:
state =[]
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'merchant_state']
    state.append(array.mode())
state[0:4]

[0    QLD
 dtype: object,
 0    NSW
 dtype: object,
 0    QLD
 dtype: object,
 0    NSW
 dtype: object]

In [58]:
# include age and gendeeeer column from riginal data

In [59]:
age =[]
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'age']
    age.append(array.mode())
age[0:4]

[0    26
 dtype: int64,
 0    38
 dtype: int64,
 0    40
 dtype: int64,
 0    20
 dtype: int64]

In [60]:
gender =[]
for id in unique_id:
    array = data.loc[data.customer_id ==id, 'gender']
    gender.append(array.mode())
gender[0:4]

[0    F
 dtype: object,
 0    M
 dtype: object,
 0    F
 dtype: object,
 0    M
 dtype: object]

In [61]:
#Predictor variadle
features_df = pd.DataFrame({'Cust_id':unique_id,'avg_no_of_weekly_trans':avg_of_weekly_trans,'max_amount':max_amount,"no_of_large_transaction":no_large_trans,'avg_trans_amount':avg_trn_amount,'Median_balance':median_balance,"State":state,"age":age,"gender":gender})
features_df[0:5]

,Cust_id,avg_no_of_weekly_trans,max_amount,no_of_large_transaction,avg_trans_amount,Median_balance,State,age,gender
0,CUS-2487424745,0,1452.21,14,45.348772,1580.40,0 QLD dtype: object,0 26 dtype: int64,0 F dtype: object
1,CUS-2142601169,0,2349.55,14,78.206106,1132.66,0 NSW dtype: object,0 38 dtype: int64,0 M dtype: object
2,CUS-1614226872,0,892.09,14,74.465019,3618.50,0 QLD dtype: object,0 40 dtype: int64,0 F dtype: object
3,CUS-2688605418,0,2320.30,14,159.304186,5616.63,0 NSW dtype: object,0 20 dtype: int64,0 M dtype: object
4,CUS-4123612273,1,1068.04,14,166.508358,6162.45,0 VIC dtype: object,0 43 dtype: int64,0 F dtype: object


In [62]:
df = pd.concat([features_df,salary_df.annual_salart],axis =1)
df

,Cust_id,avg_no_of_weekly_trans,max_amount,no_of_large_transaction,avg_trans_amount,Median_balance,State,age,gender,annual_salart
0,CUS-2487424745,0,1452.21,14,45.348772,1580.40,0 QLD dtype: object,0 26 dtype: int64,0 F dtype: object,0 52891.8525 dtype: float64
1,CUS-2142601169,0,2349.55,14,78.206106,1132.66,0 NSW dtype: object,0 38 dtype: int64,0 M dtype: object,0 52289.711786 dtype: float64
2,CUS-1614226872,0,892.09,14,74.465019,3618.50,0 QLD dtype: object,0 40 dtype: int64,0 F dtype: object,0 46547.981786 dtype: float64
3,CUS-2688605418,0,2320.30,14,159.304186,5616.63,0 NSW dtype: object,0 20 dtype: int64,0 M dtype: object,0 60534.969643 dtype: float64
4,CUS-4123612273,1,1068.04,14,166.508358,6162.45,0 VIC dtype: object,0 43 dtype: int64,0 F dtype: object,0 55728.801429 dtype: float64
...,...,...,...,...,...,...,...,...,...,...
95,CUS-2505971401,13,1946.57,14,291.293333,11410.18,0 n/a dtype: object,0 40 dtype: int64,0 F dtype: object,0 101569.241786 dtype: float64
96,CUS-2819545904,17,3231.26,14,448.658413,110017.62,0 n/a dtype: object,0 42 dtype: int64,0 M dtype: object,0 84301.265357 dtype: float64
97,CUS-3395687666,21,1757.81,14,355.400250,16758.60,0 n/a dtype: object,0 42 dtype: int64,0 M dtype: object,0 45860.007321 dtype: float64
98,CUS-1147642491,13,1711.39,14,245.600169,8473.43,0 NSW dtype: object,0 34 dtype: int64,0 F dtype: object,0 89297.885357 dtype: float64


In [63]:
#Check for missing values 

In [64]:
df.isnull().sum()

Cust_id                    0
avg_no_of_weekly_trans     0
max_amount                 0
no_of_large_transaction    0
avg_trans_amount           0
Median_balance             0
State                      0
age                        0
gender                     0
annual_salart              0
dtype: int64

In [65]:
#3.3Preprocessing the data.

In [66]:
#3.1:Train-Test split

In [67]:
x = df.drop(['Cust_id','annual_salart'],axis=1)
y = df.annual_salart
print("x Shape is ",x.shape)
print("y Shape is ",y.shape)

x Shape is  (100, 8)
y Shape is  (100,)


In [68]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size =0.3,random_state = 42)
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
print("Y_train shape", Y_train.shape)
print("Y_test shape", Y_test.shape)

X_train shape (70, 8)
X_test shape (30, 8)
Y_train shape (70,)
Y_test shape (30,)


In [69]:
# Crete column transformer 
ohe = OneHotEncoder(sparse = False)
scaler = StandardScaler()
column_transform = make_column_transformer((ohe, ["State", "gender"]), (scaler, ["avg_no_of_weekly_trans", "max_amount", "no_of_large_transaction", "avg_trans_amount", "Median_balance", "age"]))

In [70]:
#Linear Regression

In [71]:
# Instantiate model and pipeline
lm = LinearRegression()
lm_pipeline = make_pipeline(column_transform, lm)
# 

In [72]:
#Fit pipeline and make predictions 
lm_pipeline.fit(X_train, Y_train)
lm_pred = lm_pipeline.predict(X_test)
## RMSE
print("RMSE: ", round(np.sqrt(mean_squared_error(lm_pred, Y_test)))) 

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['Series']

In [ ]:
#Decision Tree

In [73]:
# Instantiate model and pipeline 
tree = DecisionTreeRegressor()
tree_pipeline = make_pipeline(column_transform, tree)
# Fit pipeline and make predictions 
tree_pipeline.fit(X_train, Y_train)
tree_pred = tree_pipeline.predict(X_test)
# RMSE 
print("RMSE: ", round(np.sqrt(mean_squared_error(tree_pred, Y_test))))

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['Series']